In [1]:
#import libraries
import requests
from bs4 import BeautifulSoup

import pandas as pd
import re


In [2]:
num = 1 #페이지 초기값
while True:
    page = requests.get('http://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2019/start/{}' .format(num)) #url의 페이지 소스를 모두 가져온다.
    soup = BeautifulSoup(page.text, 'html.parser') # BeautifulSoup으로 html을 파싱한다.
    
    #반복문 1회 때 DataFrame의 columns을 만든다
    if num == 1: 
        header = soup.find('tr', attrs={'class': 'colhead'}) #컬럼명을 가져온다.
        columns = [col.get_text() for col in header.find_all('td')] #컬럼명을 리스트로 만든다.
        final_df = pd.DataFrame(columns= columns) #DataFrame을 만들고 컬럼명을 넣는다.
        
    players = soup.find_all('tr', attrs={'class': re.compile('row player-10-')}) #선수들의 정보를 가져온다.
    
    #만약 가져온 정보가 없다면 반복문을 종료함
    if len(players) == 0: 
        break
    
    #선수들의 통계치를 DataFrame에 넣는다. 
    for player in players:
        stats = [stat.get_text() for stat in player.find_all('td')] #통계치를 가져온다.
        temp_df = pd.DataFrame(stats).transpose() #리스트 타입의 정보를 DataFrame으로 넣고 전치행렬한다.
        temp_df.columns = columns #column명을 넣는다.
        
        final_df = pd.concat([final_df, temp_df], ignore_index = True)  #최종 DataFrame과 연결한다. 
    num += 50 #페이지 번호를 50간격으로 증가시킨다.
print('Done!')

#csv 형식으로 내보내기.
final_df.to_csv(r'mlb_players_stats_2019.csv', index=False, sep=',', encoding='utf-8')

Done!
